In [13]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import string
import numpy as np
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
#from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import linear_model
from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/brechann/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/brechann/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
books = pd.read_pickle("bigger_romance.pkl")

In [8]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0


In [9]:
books.shape

(11018, 18)

In [10]:
stopWords = set(stopwords.words('english'))

In [11]:
len(stopWords)

179

##### Description words and ratings

In [14]:
descriptions=[]
for description in books['description']:
    
    descriptionFiltered=[]
    for word in word_tokenize(description.lower()):
        if word not in stopWords and word not in string.punctuation:
            descriptionFiltered.append(word)
    descriptions.append(descriptionFiltered)

In [15]:
books.loc[:,'description_bag']=descriptions

In [16]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,..."


In [17]:
books["description_frequency"]=books["description_bag"].map(lambda x: nltk.FreqDist(x))

In [18]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag,description_frequency
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ...","{'sexy': 1, 'bit': 1, 'dates': 1, 'change': 1,..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re...","{'since': 1, 'ca': 1, 'glory': 1, 'long': 1, '..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ...","{'adam': 4, 'years': 1, 'way': 1, 'gets': 1, '..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan...","{'two': 1, 'years': 2, 'way': 1, 'loving': 1, ..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,...","{'join': 1, 'way': 1, 'attraction': 1, 'months..."


In [19]:
years=books['year']

In [20]:
years=years.dropna()

In [21]:
years=years.tolist()

In [22]:
years=set(years)

In [23]:
years=list(years)

In [24]:
type(years)

list

In [25]:
years[0]

1950.0

In [26]:
my_df=[]
for year in years:
    c=Counter() 
    #year=int(year)
    #print(year)
    for index,row in books.iterrows():
        #print(row['year'])
        if row['year']==year:
            #print(index)
            c= c + row['description_frequency']
        d = {
        'year' : year,
        'freqs': c
    }
    my_df.append(d)

my_df = pd.DataFrame(my_df)

In [27]:
my_df.head()

,freqs,year
0,"{'''': 1, '``': 1, 'sentence': 1, 'deny': 1, '...",1950.0
1,"{'powerless': 1, 'difficult': 1, 'new': 1, 'wh...",1960.0
2,"{'begins': 1, 'silverstein': 1, '``': 2, 'stor...",1964.0
3,"{'''': 2, 'una': 2, 'prefìsso': 1, 'le': 2, 't...",1965.0
4,"{'napoleon': 1, 'post': 1, 'offered': 1, 'take...",1966.0


In [28]:
no_years = books[["year"]].groupby(["year"]).size()

In [29]:
no_years.head()

year
1950.0    1
1960.0    1
1964.0    1
1965.0    1
1966.0    1
dtype: int64

In [30]:
no_years.reset_index()

,year,0
0,1950.0,1
1,1960.0,1
2,1964.0,1
3,1965.0,1
4,1966.0,1
5,1970.0,2
6,1971.0,2
7,1972.0,1
8,1973.0,1
9,1974.0,3


In [31]:
counts=no_years.tolist()

In [32]:
type(no_years)

pandas.core.series.Series

In [33]:
my_df['count']=counts

In [34]:
my_df.head()

,freqs,year,count
0,"{'''': 1, '``': 1, 'sentence': 1, 'deny': 1, '...",1950.0,1
1,"{'powerless': 1, 'difficult': 1, 'new': 1, 'wh...",1960.0,1
2,"{'begins': 1, 'silverstein': 1, '``': 2, 'stor...",1964.0,1
3,"{'''': 2, 'una': 2, 'prefìsso': 1, 'le': 2, 't...",1965.0,1
4,"{'napoleon': 1, 'post': 1, 'offered': 1, 'take...",1966.0,1


In [35]:
my_df["freqs"].map(lambda x: x.get("cowboy"))

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
8      NaN
9      NaN
10     NaN
11     NaN
12     NaN
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     NaN
23     NaN
24     1.0
25     2.0
26     NaN
27     NaN
28     NaN
29     NaN
30     NaN
31     2.0
32     NaN
33     NaN
34     NaN
35     NaN
36     NaN
37     4.0
38     3.0
39     NaN
40     NaN
41     NaN
42     7.0
43     6.0
44    10.0
45    16.0
46    18.0
47    19.0
48    19.0
49    24.0
50     1.0
51     3.0
52     NaN
53     NaN
54     NaN
Name: freqs, dtype: float64

### Descriptions with scores

In [36]:
tfDescriptions = TfidfVectorizer(analyzer='word',min_df=1, 
                                 stop_words = 'english')

In [37]:
transformed_descriptions= tfDescriptions.fit_transform(books["description"])

In [38]:
transformed_descriptions

<11018x53421 sparse matrix of type '<class 'numpy.float64'>'
	with 756750 stored elements in Compressed Sparse Row format>

In [39]:
regrDescriptions = linear_model.LinearRegression()

In [40]:
regrDescriptions.fit(transformed_descriptions, books["rating"])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [41]:
resultDescriptions= pd.DataFrame({"coef":regrDescriptions.coef_,
                        "labels":tfDescriptions.get_feature_names()})

In [42]:
big_description=[word for description in descriptions for word in description]

In [43]:
corpus_frequency=nltk.FreqDist(big_description)

In [44]:
resultDescriptions["count"]=resultDescriptions["labels"].map(lambda x: corpus_frequency.get(x))

In [45]:
resultDescriptions.tail()

,coef,labels,count
53416,0.092036,自分の成績と将来にしか興味がなかった水谷雫は,NaN
53417,0.034223,軍師,1.0
53418,0.034223,転校続きの中学時代とは違い,NaN
53419,0.034223,高校生編がスタート,NaN
53420,0.004735,ﬂawless,1.0


In [ ]:
resultDescriptions.to_pickle("Descriptions_bigger.pkl")

In [46]:
resultDescriptions.sort_values("coef",inplace=True,ascending=True)

In [47]:
resultDescriptions[resultDescriptions["count"]>20].tail(100)

,coef,labels,count
41790,0.920160,sky,103.0
18854,0.922753,gates,27.0
43416,0.927973,stepping,25.0
10554,0.931794,counted,28.0
19987,0.934142,griffin,53.0
17964,0.935982,ford,48.0
28130,0.939227,manuscript,22.0
34884,0.939918,potter,28.0
37199,0.940496,regardless,32.0
30026,0.940897,moral,26.0


#### list of words for themes

In [48]:
royalty=['princess','queen','king','royalty','royal','crown', 'castle']
military=['military', 'navy', 'army', 'soldier']
supernatural=['vampire','werewolf','ghost','paranormal','supernatural','psychic','witch']
holiday=['christmas', 'thanksgiving', 'valentine', 'halloween', 'new year']
bdsm=['bdsm','dominant','submissive']
adventure=['quest','warrior','voyage','journey','adventure']
wedding=['wedding','bride','groom']
cowboy=['cowboy','cowgirl','farm']
money=['billionaire', 'rich', 'millionaire', 'wealthy']
traits=['novella','series','debut']
settings=['england','scotland','europe','asia','india','china','japan','hawaii','california','texas','hollywood',
          'ireland','florida','france']

In [49]:
all_key_words=royalty+military+supernatural+holiday+bdsm+adventure+wedding+cowboy+money+traits+settings

In [50]:
resultDescriptions[resultDescriptions['labels'].isin(holiday)]

,coef,labels,count
45601,-0.383150,thanksgiving,12.0
48361,-0.124198,valentine,68.0
20482,-0.096565,halloween,21.0
8648,0.467728,christmas,273.0


In [51]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag,description_frequency
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ...","{'sexy': 1, 'bit': 1, 'dates': 1, 'change': 1,..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re...","{'since': 1, 'ca': 1, 'glory': 1, 'long': 1, '..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ...","{'adam': 4, 'years': 1, 'way': 1, 'gets': 1, '..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan...","{'two': 1, 'years': 2, 'way': 1, 'loving': 1, ..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,...","{'join': 1, 'way': 1, 'attraction': 1, 'months..."


##### Description words vs. year

In [52]:
regrDescriptionYear = linear_model.LinearRegression()

In [53]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag,description_frequency
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ...","{'sexy': 1, 'bit': 1, 'dates': 1, 'change': 1,..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re...","{'since': 1, 'ca': 1, 'glory': 1, 'long': 1, '..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ...","{'adam': 4, 'years': 1, 'way': 1, 'gets': 1, '..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan...","{'two': 1, 'years': 2, 'way': 1, 'loving': 1, ..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,...","{'join': 1, 'way': 1, 'attraction': 1, 'months..."


In [54]:
transformed_descriptions_year=tfDescriptions.transform(books.loc[~(books["year"].isnull()),'description'])

In [55]:
regrDescriptionYear.fit(transformed_descriptions_year, 
                        books.loc[~(books["year"].isnull()),'year'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [56]:
resultDescriptionYear= pd.DataFrame({"coef":regrDescriptionYear.coef_,"labels":tfDescriptions.get_feature_names()})

In [57]:
resultDescriptionYear.to_pickle("Description_Year.pkl")

In [58]:
resultDescriptionYear.sort_values("coef",inplace=True,ascending=True)

In [59]:
resultDescriptionYear["count"]=resultDescriptionYear["labels"].map(lambda x: corpus_frequency.get(x))

In [60]:
resultDescriptionYear[resultDescriptionYear["count"]>30]

,coef,labels,count
3303,-115.943200,asin,119.0
23837,-103.227584,isbn,61.0
8893,-19.998951,classic,121.0
40542,-19.962746,sensuous,36.0
19414,-19.606547,glamorous,79.0
35711,-18.719544,proud,103.0
17246,-18.427963,fiance,52.0
27254,-17.663776,lovely,98.0
42438,-17.353573,sophisticated,42.0
50514,-17.081930,worries,38.0


In [61]:
resultDescriptionYear[resultDescriptionYear['labels'].isin(military)]

,coef,labels,count
42296,-8.206803,soldier,98.0
29413,0.573264,military,129.0
3098,2.200788,army,151.0
30807,2.326047,navy,76.0


##### Review words vs. ratings 

Redoing review words vs ratings so can pick apart the rating a specific reviewer gave with their specific review

In [62]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag,description_frequency
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ...","{'sexy': 1, 'bit': 1, 'dates': 1, 'change': 1,..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re...","{'since': 1, 'ca': 1, 'glory': 1, 'long': 1, '..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ...","{'adam': 4, 'years': 1, 'way': 1, 'gets': 1, '..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan...","{'two': 1, 'years': 2, 'way': 1, 'loving': 1, ..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,...","{'join': 1, 'way': 1, 'attraction': 1, 'months..."


In [63]:
les_review=books['review_texts'].tolist()

In [64]:
len(les_review)

11018

In [65]:
newlist=[]
for i in range(0,11018):
    #print(i)
    x=les_review[i]
    for j in range(0,len(x)):
        #print(j)
        try:
            y=x[j][0]
        except:
            y=''
        newlist.append(y)

In [66]:
len(newlist)

1331878

In [67]:
#all_reviews=[item for sublist in flat_list for item in sublist]

In [68]:
all_scores=[score for scores in books['reviewer_scores'] for score in scores]

In [69]:
len(all_scores)

1331878

In [70]:
scores_reviews=pd.DataFrame(
    {'reviews': newlist,
     'scores': all_scores
    })

In [71]:
no_blanks=scores_reviews[scores_reviews["scores"]!='']

In [72]:
no_blanks.shape

(1322191, 2)

In [73]:
les_scores=no_blanks['scores'].tolist()

In [74]:
len(les_scores)

1322191

In [75]:
tfReviews = TfidfVectorizer(analyzer='word',min_df=0.001, stop_words = 'english')

In [76]:
transformed_reviews= tfReviews.fit_transform(no_blanks["reviews"])

In [77]:
transformed_reviews.shape

(1322191, 4978)

In [78]:
regrReviews = linear_model.LinearRegression()

In [79]:
regrReviews.fit(transformed_reviews, les_scores)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [80]:
resultReviews= pd.DataFrame({"coef":regrReviews.coef_,"labels":tfReviews.get_feature_names()})

In [81]:
resultReviews.head()

,coef,labels
0,0.248113,10
1,-0.030964,100
2,0.210384,11
3,-0.276193,12
4,-0.243209,13


In [82]:
len(resultReviews)

4978

In [ ]:
resultReviews.to_pickle('bigger_reviews_split.pkl')

In [83]:
blah=resultReviews.sort_values('coef')

In [84]:
sorted_words=blah['labels'].tolist()

In [86]:
#for x in range(4700,4978):
       # print(sorted_words[x])

In [87]:
resultReviews[resultReviews['labels'].isin(money)]

,coef,labels
462,0.115753,billionaire
3662,0.042821,rich
4806,-0.325337,wealthy


In [88]:
resultReviews[resultReviews['labels'].isin(military)]

,coef,labels
269,0.157777,army
2824,-0.007490,military
2948,-0.275866,navy
4054,-0.028246,soldier


In [89]:
resultReviews[resultReviews['labels'].isin(holiday)]

,coef,labels
745,-0.003684,christmas
4701,0.090408,valentine


In [90]:
resultReviews[resultReviews['labels'].isin(settings)]

,coef,labels
610,0.312965,california
1416,-0.013701,england
1759,0.039427,france
2102,-0.059196,hollywood
2229,-0.143851,india
2304,-0.040700,ireland
3804,-0.024269,scotland
4428,0.148300,texas


In [91]:
resultReviews[resultReviews['labels'].isin(supernatural)]

,coef,labels
1834,-0.084401,ghost
3140,0.021839,paranormal
3407,-0.377121,psychic
4285,-0.128185,supernatural
4704,-0.123867,vampire
4828,0.022883,werewolf
4871,-0.160253,witch


In [92]:
resultReviews[resultReviews['labels'].isin(adventure)]

,coef,labels
111,0.384271,adventure
2373,0.437349,journey
3457,0.031580,quest
4789,0.157107,warrior


In [93]:
resultReviews[resultReviews['labels'].isin(traits)]

,coef,labels
1058,0.137533,debut
3017,-0.125440,novella
3883,0.182261,series


### Reviews individually with their dates

In [94]:
books.head()

,authors,awards,date,description,isbn,link,num_ratings,num_reviews,rating,review_dates,review_ids,review_texts,reviewer_names,reviewer_ratings,reviewer_scores,reviews_together,title,year,description_bag,description_frequency
40,[Jennifer Crusie],[All About Romance (AAR) Annual Reader Poll fo...,December 1st 2006,"For Nina Askew, turning forty means freedom--f...",037377138X,http://www.goodreads.com/book/show/33732.Anyon...,17661,928,3.8,"[Mar 11, 2010, Sep 20, 2009, Feb 27, 2009, Dec...","[review_93605589, review_71904939, review_4772...",[[Opening Line: “The last thing Nina Askew nee...,"[Buggy, Shawna, Kathrynn, Ⓐlleskelle - teamSør...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 5, 3, 5, 5, 4, 3, 4, 4, 5, 5, 3, 4, ...",Opening Line: “The last thing Nina Askew neede...,Anyone But You,2006.0,"[nina, askew, turning, forty, means, freedom, ...","{'sexy': 1, 'bit': 1, 'dates': 1, 'change': 1,..."
45,[Rachel Gibson],[OKRWA National Readers Choice Award for Singl...,January 1st 1998,Georgeanne Howard leaves her fiancé at the alt...,0380790076,http://www.goodreads.com/book/show/60220.Simpl...,18156,635,3.86,"[Oct 20, 2010, Sep 30, 2009, Apr 04, 2017, Jun...","[review_127198135, review_73023830, review_195...",[[3.5 STARSI must admit I was a bit disappoint...,"[valee, Auntee, Nenia *The Flagrant Liberal* C...","[liked it, it was amazing, it was ok, it was o...","[3, 5, 2, 2, 4, 4, 3, 4, 4, 5, 4, 5, 4, 3, 5, ...",3.5 STARSI must admit I was a bit disappointed...,Simply Irresistible\n \n (Chinoo...,1998.0,"[georgeanne, howard, leaves, fiancé, altar, re...","{'since': 1, 'ca': 1, 'glory': 1, 'long': 1, '..."
59,[Kallypso Masters],[],December 13th 2012,Librarian's Note: This is an alternate cover f...,1481152718,http://www.goodreads.com/book/show/17325954-no...,11078,579,4.31,"[Apr 20, 2013, Nov 28, 2011, Nov 14, 2012, Jun...","[review_595540086, review_240655375, review_45...",[[1 star. DNF at 52 %*Emotional porn at its wo...,"[Baba ♥♥♥ Tyler, Marcus, Archer, Dean, Adrian...","[did not like it, it was amazing, it was ok, r...","[1, 5, 2, 4, 5, 1, 5, 2, 4, 5, 5, 5, 5, 5, 5, ...","[ in the hospital (hide spoiler)] Ka-Thunk ,Ka...",Nobody's Hero\n \n (Rescue Me Sa...,2012.0,"[librarian, 's, note, alternate, cover, asin, ...","{'adam': 4, 'years': 1, 'way': 1, 'gets': 1, '..."
61,[Kallypso Masters],[],January 24th 2013,NOBODY'S PERFECT is the continuing story of Sa...,1480096954,http://www.goodreads.com/book/show/17299927-no...,8462,506,4.34,"[Jun 09, 2012, Feb 25, 2012, May 28, 2012, Mar...","[review_345240483, review_283985473, review_33...","[[Nobody's perfect, mi sueño, but you're about...","[~ Becs ~, CaroleDee, Nia, Becca, Amy, Michael...","[really liked it, it was amazing, it was amazi...","[4, 5, 5, 2, 2, 5, 2, 5, 4, 5, 1, 5, 3, 5, 4, ...","Nobody's perfect, mi sueño, but you're about a...",Nobody's Perfect\n \n (Rescue Me...,2013.0,"[nobody, 's, perfect, continuing, story, savan...","{'two': 1, 'years': 2, 'way': 1, 'loving': 1, ..."
66,[Kallypso Masters],[],December 22nd 2013,When Marc d’Alessio first rescued the curvaceo...,NA,http://www.goodreads.com/book/show/17207097-so...,3940,331,4.28,"[Jan 04, 2013, Nov 12, 2013, Jan 02, 2014, Dec...","[review_494970690, review_763151413, review_80...","[[I like this series, but this one was a littl...","[Vivian, Sylvia, Morgan, Sue, Rebekah, Dianne,...","[liked it, did not like it, it was ok, really ...","[3, 1, 2, 4, 2, 4, 2, 4, 2, 5, 5, 5, 4, 5, 4, ...",It's LIVE! It's live! It's finally live! OMG! ...,Somebody's Angel\n \n (Rescue Me...,2013.0,"[marc, ’, alessio, first, rescued, curvaceous,...","{'join': 1, 'way': 1, 'attraction': 1, 'months..."


In [95]:
all_dates=[date for dates in books['review_dates'] for date in dates]

In [96]:
len(all_dates)

1331878

In [97]:
len(newlist)

1331878

In [98]:
dates_reviews=pd.DataFrame(
    {'reviews': newlist,
     'dates': all_dates
    })

In [99]:
dates_reviews.head()

,dates,reviews
0,"Mar 11, 2010",Opening Line: “The last thing Nina Askew neede...
1,"Sep 20, 2009",5 stars – Contemporary Romance40-year-old divo...
2,"Feb 27, 2009",Loved it! I have a new favorite author. :-) Th...
3,"Dec 10, 2014",⭐️⭐️⭐️⭐️⭐️ 5 STARS⭐️⭐️⭐️⭐️⭐️Jennifer Crusie ca...
4,"Jan 14, 2016","With her fairytale marriage-gone-bust, Nina As..."


In [100]:
dates_reviews["dts"]=pd.to_datetime(dates_reviews["dates"])

In [101]:
dates_reviews.head()

,dates,reviews,dts
0,"Mar 11, 2010",Opening Line: “The last thing Nina Askew neede...,2010-03-11
1,"Sep 20, 2009",5 stars – Contemporary Romance40-year-old divo...,2009-09-20
2,"Feb 27, 2009",Loved it! I have a new favorite author. :-) Th...,2009-02-27
3,"Dec 10, 2014",⭐️⭐️⭐️⭐️⭐️ 5 STARS⭐️⭐️⭐️⭐️⭐️Jennifer Crusie ca...,2014-12-10
4,"Jan 14, 2016","With her fairytale marriage-gone-bust, Nina As...",2016-01-14


In [102]:
dates_reviews['date_formatted'] = dates_reviews['dts'].dt.strftime('%b-%Y')

In [103]:
dates_reviews.head()

,dates,reviews,dts,date_formatted
0,"Mar 11, 2010",Opening Line: “The last thing Nina Askew neede...,2010-03-11,Mar-2010
1,"Sep 20, 2009",5 stars – Contemporary Romance40-year-old divo...,2009-09-20,Sep-2009
2,"Feb 27, 2009",Loved it! I have a new favorite author. :-) Th...,2009-02-27,Feb-2009
3,"Dec 10, 2014",⭐️⭐️⭐️⭐️⭐️ 5 STARS⭐️⭐️⭐️⭐️⭐️Jennifer Crusie ca...,2014-12-10,Dec-2014
4,"Jan 14, 2016","With her fairytale marriage-gone-bust, Nina As...",2016-01-14,Jan-2016


In [ ]:
#dates_reviews.to_pickle('bigger_reviews_years.pkl')

In [14]:
#dates_reviews = pd.read_pickle("bigger_reviews_years.pkl")

In [ ]:
blah=dates_reviews[['date_formatted','reviews']].groupby('date_formatted').sum()

In [ ]:
#blah=dates_reviews[['reviews']].groupby('date_formatted').apply(lambda x: x.sum())

In [ ]:
blah.head()

In [7]:
#blah.to_pickle('reviews_month_year.pkl')

In [5]:
#blah=pd.read_pickle("reviews_month_year.pkl")

In [6]:
review_dates=[]
for review in blah['reviews']:
    
    reviewFiltered=[]
    for word in word_tokenize(review.lower()):
        if word in all_key_words:
            reviewFiltered.append(word)
    review_dates.append(reviewFiltered)

In [ ]:
review_dates=[]
for review in blah['reviews']:
    
    reviewFiltered=[]
    for word in word_tokenize(review.lower()):
        if word not in stopWords and word not in string.punctuation:
            reviewFiltered.append(word)
    review_dates.append(reviewFiltered)

In [1]:
len(review_dates)

NameError: name 'review_dates' is not defined

In [ ]:
len(blah)

In [9]:
blah['review_bag']=review_dates

In [10]:
blah.head()

,reviews,review_bag
date_formatted,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france..."
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para..."
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,..."
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ..."
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri..."


In [11]:
blah["review_date_frequency"]=blah["review_bag"].map(lambda x: nltk.FreqDist(x))

In [12]:
blah.head()

,reviews,review_bag,review_date_frequency
date_formatted,,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france...","{'rich': 1, 'series': 9, 'england': 1, 'france..."
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para...","{'series': 105, 'wedding': 15, 'paranormal': 3..."
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,...","{'series': 180, 'california': 3, 'military': 3..."
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ...","{'millionaire': 3, 'series': 258, 'debut': 3, ..."
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri...","{'series': 665, 'wedding': 43, 'europe': 9, 'j..."


In [16]:
num_months=dates_reviews [["date_formatted"]].groupby(["date_formatted"]).size()

In [17]:
num_months.reset_index()

,date_formatted,0
0,Apr-2007,28
1,Apr-2008,323
2,Apr-2009,442
3,Apr-2010,752
4,Apr-2011,1911
5,Apr-2012,4304
6,Apr-2013,12676
7,Apr-2014,16866
8,Apr-2015,18228
9,Apr-2016,19759


In [18]:
counts_mo=num_months.tolist()

In [19]:
type(counts_mo)

list

In [20]:
blah['counts']=counts_mo

In [21]:
blah.head()

,reviews,review_bag,review_date_frequency,counts
date_formatted,,,,
Apr-2007,Quinton Murphy knows what he likes and what he...,"[rich, series, series, series, england, france...","{'rich': 1, 'series': 9, 'england': 1, 'france...",28
Apr-2008,The 4th and final book in the Chesapeake Bay /...,"[series, series, series, wedding, series, para...","{'series': 105, 'wedding': 15, 'paranormal': 3...",323
Apr-2009,A great story filled with romance and tenderne...,"[series, california, series, military, series,...","{'series': 180, 'california': 3, 'military': 3...",442
Apr-2010,It did nothing for me. The aspect of 'small to...,"[millionaire, series, series, series, series, ...","{'millionaire': 3, 'series': 258, 'debut': 3, ...",752
Apr-2011,Simple Irresistible is the first book in the C...,"[series, series, series, series, wedding, seri...","{'series': 665, 'wedding': 43, 'europe': 9, 'j...",1911


In [23]:
blah.to_pickle("review_word_frequency_by_month_bigger.pkl")

In [ ]:
blah["review_date_frequency"].map(lambda x: x.get("cowboy"))

Old way (all smushed together)

In [ ]:
books["reviews_together"]= books["reviews"].map(lambda x:" ".join(x))  

In [ ]:
tfReviews = TfidfVectorizer(analyzer='word',min_df=1, stop_words = 'english')

In [ ]:
transformed_reviews= tfReviews.fit_transform(books["reviews_together"])

In [ ]:
transformed_reviews

In [ ]:
regrReviews = linear_model.LinearRegression()

In [ ]:
regrReviews.fit(transformed_reviews, books["rating"])

In [ ]:
resultReviews= pd.DataFrame({"coef":regrReviews.coef_,"labels":tfReviews.get_feature_names()})

In [ ]:
resultReviews.head()

In [ ]:
resultReviews["count"]=resultReviews["labels"].map(lambda x: corpus_frequency.get(x))

In [ ]:
resultReviews.to_pickle("Reviews_ratings.pkl")

In [ ]:
resultReviews.sort_values("coef",inplace=True,ascending=True)

In [ ]:
resultReviews.head()

In [ ]:
resultReviews[resultReviews["count"]>10]

In [ ]:
sortthis = resultReviews.sort_values("count",ascending=False)

In [ ]:
sortthis.head(50)